In [ ]:
ONE-PAGE REAL-TIME TROUBLESHOOTING GUIDE
Windows UF + Linux UF + Linux Syslog
1. WINDOWS UNIVERSAL FORWARDER (UF) – REAL-TIME TROUBLESHOOTING
1. Check Forwarder Service
services.msc → SplunkForwarder = Running


Reason: If the service stopped, no events will forward.

2. Check UF Internal Logs
C:\Program Files\SplunkUniversalForwarder\var\log\splunk\splunkd.log


Reason: Shows connection errors, permission issues, or misconfigurations.

3. Validate Connectivity to Indexer (9997)
Test-NetConnection -ComputerName <indexer_ip> -Port 9997


Reason: Confirms firewall/network is not blocking forwarding.

4. Check WinEventLog Permissions

UF user must read:

Security

System

Application
Reason: Lack of read permission → No logs in Splunk.

5. Validate Configuration Files

inputs.conf

[WinEventLog://Security]
disabled = 0


outputs.conf

server = <indexer1>:9997,<indexer2>:9997


Reason: Wrong index, wrong server, or disabled input stops data.

6. Validate Data in Splunk
index=wineventlog host=<hostname> | head 20


Reason: Confirms if data is reaching Splunk.

2. LINUX UNIVERSAL FORWARDER (UF) – REAL-TIME TROUBLESHOOTING
1. Check UF Process
ps -ef | grep splunkd
/opt/splunkforwarder/bin/splunk status


Reason: splunkd must be running to forward logs.

2. Validate Port 9997 Connectivity
nc -zv <indexer_ip> 9997


Reason: Firewall or network blocks stop forwarding.

3. Validate File Read Permissions
ls -l /var/log/messages
ls -l /var/log/secure


Reason: UF can’t forward unreadable log files.

4. Confirm Logs Are Updating
tail -f /var/log/messages


Reason: If log not updating, UF has nothing to send.

5. Validate UF Configurations

inputs.conf

[monitor:///var/log/messages]
index = linux


outputs.conf

server = indexer1:9997


Reason: Wrong path/index/output causes data loss.

6. Examine UF Logs
tail -f /opt/splunkforwarder/var/log/splunk/splunkd.log


Reason: Shows S2S failures, connection reset, or read errors.

7. Verify ulimit
ulimit -n


Recommended: 65535
Reason: Low limits break file monitoring.

3. LINUX SYSLOG (rsyslog / syslog-ng) – REAL-TIME TROUBLESHOOTING
1. Check Syslog Service
systemctl status rsyslog
journalctl -u rsyslog | tail


Reason: Restart loops or errors → no forwarding.

2. Verify Syslog Forwarding Configuration
cat /etc/rsyslog.d/50-splunk.conf


UDP:

*.* @<splunk_receiver>:514


TCP:

*.* @@<splunk_receiver>:514


TLS:

*.* @@(o) <splunk_receiver>:6514


Reason: Incorrect format or wrong IP → no logs.

3. Test Connectivity
nc -zv <receiver_ip> 514
nc -zv <receiver_ip> 6514


Reason: Firewall commonly blocks syslog.

4. Check SELinux/AppArmor
getenforce
ausearch -m avc


Reason: SELinux can silently block rsyslog from sending logs.

5. Check if Logs Are Leaving the Server
tcpdump -nn host <splunk_receiver> and port 514


Reason: If no packets, forwarding is not happening.

6. Validate Splunk Receiver

On Heavy Forwarder:

netstat -tulnp | grep 514
tail -f /opt/splunk/var/log/splunk/splunkd.log | grep syslog


Reason: Confirms receiver active & no parsing issues.

7. Verify Data in Splunk
index=linux sourcetype=syslog host=<hostname> | head 20


Reason: Confirms ingestion end-to-end.

8. Check Log Rotation
ls -l /var/log
cat /etc/logrotate.conf


Reason: Rotation can remove/rename logs before rsyslog sends them.

🔥 FINAL REAL-TIME SUMMARY (What to check first)
Windows UF

✔ Service running
✔ outputs.conf → 9997
✔ EventLog permissions
✔ Firewall open
✔ splunkd.log errors

Linux UF

✔ splunkd running
✔ Connectivity to 9997
✔ Log file permissions
✔ Log files updating
✔ inputs.conf / outputs.conf
✔ ulimit value

Linux Syslog

✔ rsyslog running
✔ Syslog config correct
✔ Port 514/6514 reachable
✔ SELinux not blocking
✔ tcpdump shows traffic
✔ HF listening
✔ events seen in Splunk